# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps as maps
import os

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
weather_df = pd.read_csv("../output_data.csv")

weather_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Conde,75,BR,74,-7.26,-34.91,82.40,13.87
1,New Norfolk,99,AU,64,-42.78,147.06,55.99,1.01
2,Busselton,0,AU,77,-33.65,115.33,54.00,1.01
3,Avera,90,US,66,33.19,-82.53,48.20,11.41
4,Nome,90,US,92,64.50,-165.41,19.40,18.34
...,...,...,...,...,...,...,...,...
572,Palana,95,RU,93,59.12,159.97,14.99,15.70
573,Basco,100,PH,84,20.45,121.97,74.82,28.27
574,Dafeng,100,CN,63,23.45,108.59,42.87,7.65
575,Hengyang,100,CN,93,26.89,112.61,33.51,8.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
#gmaps
gmaps.configure(api_key=g_key)

In [50]:
locations = weather_df[["Lat","Lng"]]
humid = weather_df["Humidity"]

In [51]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating=False, max_intensity=50,point_radius= 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
city_weather = weather_df.loc[(weather_df'Max Temp >=70')]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
